# Руководство: Подготовка данных осциллограмм с помощью модульных функций

Этот Jupyter Notebook демонстрирует последовательный процесс обработки и подготовки данных осциллограмм с использованием рефакторенных функций из пакета `osc_tools`.

**Цель**: Показать, как шаг за шагом применять различные функции для очистки, стандартизации и анализа конфигурационных файлов осциллограмм (.cfg).

**Рассматриваемые шаги:**
1.  Удаление конфиденциальной информации и переименование файлов на основе хеша (используя `deleting_confidential_information_in_all_files`).
2.  Замена даты изменения файлов (используя `date_of_change_replacement`).
3.  Группировка файлов по частоте дискретизации и частоте сети (используя `grouping_by_sampling_rate_and_network`).
4.  Сбор всех уникальных имен аналоговых и цифровых сигналов (используя `find_all_name_analog_signals` и `find_all_name_digital_signals`).
5.  Ручная стандартизация имен (описание процесса).
6.  Переименование аналоговых и цифровых сигналов на основе стандартизированных имен (используя `rename_analog_signals` и `rename_digital_signals`).
7.  Поиск дубликатов имен сигналов в файлах конфигурации.

**Важно**: Примеры в этом ноутбуке предполагают наличие файлов `.cfg` и соответствующих им `.dat` файлов. Для выполнения примеров кода вам может потребоваться создать демонстрационные файлы и каталоги, как указано в ячейках кода, или адаптировать пути к вашим существующим данным.

In [ ]:
import sys
import os

# Получаем текущий рабочий каталог ноутбука
notebook_dir = os.getcwd()

# Выход на исходный коталог
project_root = os.path.abspath(os.path.join(notebook_dir, os.pardir))

sys.path.append(project_root)
print(f"Добавлен путь в sys.path: {project_root}")

In [ ]:
import os
import shutil
import json
import datetime
import csv

# --- Импорт рефакторенных функций ---
from osc_tools.data_management.processing import (
    deleting_confidential_information_in_all_files,
    date_of_change_replacement,
    grouping_by_sampling_rate_and_network,
    extract_frequencies,
    combining_json_hash_table
)
from osc_tools.data_management.renaming import (
    find_all_name_analog_signals,
    find_all_name_digital_signals,
    rename_analog_signals,
    rename_digital_signals,
    rename_one_signals,
    combining_databases_of_unique_codes
)

print("Необходимые функции импортированы.")

# --- Базовые пути для демонстрационных данных ---
# Эти пути будут использоваться в примерах ниже.
# Вы можете изменить их или создать эти каталоги и файлы вручную.

# Каталог для исходных .cfg файлов (до обработки)
# !!! ВАЖНО !!!
# Файлы в данной директории как раз будут меняться. Поэтому это должна быть копия для обработки.
# !!! ВАЖНО !!!
base_source_cfg_dir = os.path.join(project_root, "raw_data","002_run_processing_oscillograms_guide","")

os.makedirs(base_source_cfg_dir, exist_ok=True)

print("Начальная настройка завершена. Можно приступать к демонстрации функций.")

## 1. Удаление конфиденциальной информации и переименование файлов

Функция `deleting_confidential_information_in_all_files` выполняет две основные задачи:
1.  **Удаление локальной информации**: В каждой строке файла `.cfg`, содержащей информацию о станции и номере устройства (обычно первая строка), эта информация заменяется на пустые значения (например, `",,"`). Даты начала и конца записи также заменяются на `01/01/0001,01:01:01.000000`.
2.  **Переименование файлов**: После обработки файла `.cfg`, он и соответствующий ему файл `.dat` переименовываются на основе MD5-хеша содержимого файла `.dat`. Например, `original_name.cfg` и `original_name.dat` станут `[hash_value].cfg` и `[hash_value].dat`.

Этот шаг важен для анонимизации данных перед их дальнейшим использованием или публикацией.

**Параметры:**
*   `source_dir` (str): Путь к каталогу, содержащему файлы `.cfg` и `.dat` для обработки.

**Побочные эффекты:**
*   Изменяет содержимое файлов `.cfg` в `source_dir`.
*   Переименовывает файлы `.cfg` и `.dat` в `source_dir`.
*   Создает файл `protected_files.txt` в `source_dir`, содержащий список файлов, которые не удалось обработать (например, из-за проблем с кодировкой), и ошибки обработки.

In [ ]:
# --- 1. Демонстрация deleting_confidential_information_in_all_files ---

# Используем ранее определенный base_source_cfg_dir
print(f"Каталог для этого шага: {os.path.abspath(base_source_cfg_dir)}")

# Вызов функции
print("\nВызов deleting_confidential_information_in_all_files...")
deleting_confidential_information_in_all_files(source_dir=base_source_cfg_dir)
print("Функция deleting_confidential_information_in_all_files завершена.")

## 2. Замена даты изменения файлов

Функция `date_of_change_replacement` обновляет системную дату последнего изменения для всех файлов в указанном каталоге. Каждому файлу присваивается текущая системная дата и время на момент выполнения этой функции.

Это может быть полезно для стандартизации метаданных файлов или для того, чтобы отметить этап обработки.

**Параметры:**
*   `source_dir` (str): Путь к каталогу, содержащему файлы, у которых нужно обновить дату изменения.

**Побочные эффекты:**
*   Изменяет системную дату последнего изменения файлов в `source_dir`.

In [ ]:
# --- 2. Демонстрация date_of_change_replacement ---

print(f"Каталог для этого шага: {os.path.abspath(base_source_cfg_dir)}")

# Вызов функции
print("\nВызов date_of_change_replacement...")

# Сохраним текущее время ДО вызова, чтобы примерно понимать, какое время будет установлено
time_before_call = datetime.datetime.now()
date_of_change_replacement(source_dir=base_source_cfg_dir)
print("Функция date_of_change_replacement завершена.")
print(f"(Ожидаемое время изменения примерно: {time_before_call.strftime('%Y-%m-%d %H:%M:%S')} или чуть позже)")

## 3. Группировка по частоте дискретизации и сети

Функция `grouping_by_sampling_rate_and_network` анализирует файлы `.cfg` в указанном исходном каталоге, извлекает из них информацию о частоте сети (f_network) и частоте дискретизации (f_rate). Затем он создает подкаталоги в этом же исходном каталоге, именуя их по формату `f_network = [значение] and f_rate = [значение]`, и перемещает соответствующие файлы `.cfg` и их парные `.dat` файлы в эти подкаталоги.

Это помогает организовать осциллограммы по их техническим характеристикам.

**Параметры:**
*   `source_dir` (str): Путь к каталогу, содержащему файлы `.cfg` и `.dat` для группировки. Файлы будут перемещены из этого каталога в созданные в нем же подкаталоги.
*   `threshold` (float, опционально, по умолчанию 0.1): Порог для определения отклонения частоты от целого числа как ошибки измерения.
*   `isPrintMessege` (bool, опционально, по умолчанию `False`): Печатать ли сообщения, если частоты не найдены.

**Побочные эффекты:**
*   Создает новые подкаталоги внутри `source_dir`.
*   Перемещает файлы `.cfg` и `.dat` из `source_dir` в созданные подкаталоги.

In [ ]:
# --- 3. Демонстрация grouping_by_sampling_rate_and_network ---

print(f"Каталог для этого шага: {os.path.abspath(base_source_cfg_dir)}")

# Вызов функции
print("\nВызов grouping_by_sampling_rate_and_network...")
grouping_by_sampling_rate_and_network(source_dir=base_source_cfg_dir, isPrintMessege=True) # isPrintMessege=True для отладки
print("Функция grouping_by_sampling_rate_and_network завершена.")

print(f"\nСтруктура каталога '{base_source_cfg_dir}' после группировки:")
for root_dir, sub_dirs, files_in_dir in os.walk(base_source_cfg_dir):
    # Отступ для отображения иерархии
    level = root_dir.replace(base_source_cfg_dir, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root_dir)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files_in_dir:
        print(f"{sub_indent}{f_name}")


## 4. Сбор всех уникальных имен аналоговых и цифровых сигналов

На этом этапе используются две функции для сбора всех уникальных имен сигналов из файлов `.cfg` в указанном каталоге:
*   `find_all_name_analog_signals`: Собирает имена аналоговых сигналов.
*   `find_all_name_digital_signals`: Собирает имена цифровых (дискретных) сигналов.

Каждая из этих функций создает CSV-файл в том же каталоге (`source_dir`), содержащий два столбца: `Key` (имя сигнала) и `Value` (частота встречаемости этого имени во всех обработанных файлах), а также столбец `universal_code` для последующего заполнения стандартизированным именем. Файлы называются `sorted_analog_signals_name.csv` и `sorted_digital_signals_name.csv` соответственно.

**Параметры (для обеих функций):**
*   `source_dir` (str): Путь к каталогу, содержащему файлы `.cfg` для анализа. CSV-файлы с результатами будут сохранены в этот же каталог.

**Побочные эффекты:**
*   Создает CSV-файл (`sorted_analog_signals_name.csv` или `sorted_digital_signals_name.csv`) в `source_dir`.

In [ ]:
# --- 4. Демонстрация find_all_name_analog_signals и find_all_name_digital_signals ---

print(f"Каталог для этого шага: {os.path.abspath(base_source_cfg_dir)}")

# Вызов find_all_name_analog_signals
print("\nВызов find_all_name_analog_signals...")
find_all_name_analog_signals(source_dir=base_source_cfg_dir)
analog_csv_path = os.path.join(base_source_cfg_dir, "sorted_analog_signals_name.csv")
print(f"Функция find_all_name_analog_signals завершена. Результаты должны быть в: {analog_csv_path}")

if os.path.exists(analog_csv_path):
    print(f"Содержимое '{analog_csv_path}' (первые несколько строк):")
    try:
        with open(analog_csv_path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i < 5: print(f"  {line.strip()}")
                else: break
            if i >=4: print("  ...")
    except Exception as e:
        print(f"  Не удалось прочитать файл: {e}")
else:
    print(f"  Файл '{analog_csv_path}' не найден.")

# Вызов find_all_name_digital_signals
print("\nВызов find_all_name_digital_signals...")
find_all_name_digital_signals(source_dir=base_source_cfg_dir)
digital_csv_path = os.path.join(base_source_cfg_dir, "sorted_digital_signals_name.csv")
print(f"Функция find_all_name_digital_signals завершена. Результаты должны быть в: {digital_csv_path}")

if os.path.exists(digital_csv_path):
    print(f"Содержимое '{digital_csv_path}' (первые несколько строк):")
    try:
        with open(digital_csv_path, "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i < 5: print(f"  {line.strip()}")
                else: break
            if i >=4: print("  ...")
    except Exception as e:
        print(f"  Не удалось прочитать файл: {e}")
else:
    print(f"  Файл '{digital_csv_path}' не найден.")


### Важное примечание: Ручная стандартизация имен сигналов

После того как вы собрали все уникальные имена аналоговых и цифровых сигналов с помощью `find_all_name_analog_signals` и `find_all_name_digital_signals`, следующим критически важным шагом является **ручная стандартизация этих имен**.

**Процесс включает в себя:**
1.  **Анализ CSV-файлов**: Внимательно изучите сгенерированные файлы `sorted_analog_signals_name.csv` и `sorted_digital_signals_name.csv`. Обратите внимание на разнообразие имен, возможные опечатки, разные стили именования одного и того же физического сигнала.
2.  **Создание словаря замен**: Для каждого уникального имени сигнала (столбец `Key`) необходимо определить его стандартизированное, "универсальное" имя. Это универсальное имя вносится в столбец `universal_code` в этих CSV-файлах.
    *   Если для какого-то имени не удается подобрать универсальный код или оно должно быть проигнорировано, можно оставить значение по умолчанию (`-` или `?`).
3.  **Использование стандартных имен**: В вашем проекте могут существовать предопределенные стандартные имена или правила их формирования (например, `U | BusBar-1 | phase:A`). Постарайтесь придерживаться этих стандартов. В кодовой базе проекта (возможно, не в этом классе) могут быть генераторы или списки таких стандартных имен, которые могут помочь в этом процессе.
4.  **Сохранение CSV**: После заполнения столбца `universal_code` эти CSV-файлы будут использоваться функциями `rename_analog_signals` и `rename_digital_signals` для фактического переименования сигналов в файлах `.cfg`.

**Пример строки в `sorted_analog_signals_name.csv` после ручной обработки:**
```csv
Key,universal_code,Value
"Ua_System1 | phase:A","U | System-1 | phase:A",3
"Ia_System1 | phase:A","I | System-1 | phase:A",1
"Voltage_X1","U | SubstationX-Line1 | phase:A",1
...
```
Этот этап требует внимательности и хорошего понимания структуры данных и стандартов именования в вашем проекте, так как от него зависит качество и согласованность данных для последующего анализа и обучения моделей.

## 5. Переименование аналоговых и цифровых сигналов

После того как CSV-файлы с уникальными именами сигналов (`sorted_analog_signals_name.csv` и `sorted_digital_signals_name.csv`) были созданы и столбец `universal_code` в них был вручную заполнен стандартизированными именами, можно приступать к фактическому переименованию сигналов в файлах `.cfg`.

Для этого используются две функции:
*   `rename_analog_signals`: Переименовывает аналоговые сигналы.
*   `rename_digital_signals`: Переименовывает цифровые (дискретные) сигналы.

Обе функции читают соответствующий CSV-файл, создают карту замен (старое имя -> универсальное имя) и применяют эти замены к именам сигналов в каждом файле `.cfg` в указанном каталоге.

**Параметры (для обеих функций):**
*   `source_dir` (str): Путь к каталогу, содержащему файлы `.cfg`, в которых нужно переименовать сигналы.
*   `csv_dir` (str): Путь к CSV-файлу, содержащему сопоставление старых имен с универсальными кодами (например, `sorted_analog_signals_name.csv`).
*   `encoding` (str, опционально, по умолчанию 'utf8'): Кодировка CSV-файла.
*   `delimiter` (str, опционально, по умолчанию ','): Разделитель в CSV-файле.

**Побочные эффекты:**
*   Изменяет имена сигналов непосредственно в файлах `.cfg` в каталоге `source_dir`.

Внимание, проверьте 
*   `encoding` (str, опционально, по умолчанию 'utf8'): Кодировка CSV-файла.
*   `delimiter` (str, опционально, по умолчанию ','): Разделитель в CSV-файле.

Часто бывает ошибка в них и требуется либо задать верные, либо пересохранить файл.

In [ ]:
# ПРОВЕРИТЬ!
# В конце проверить ещё эти сигналы, есть ли они? И если нет, то добавить описание

# --- 5. Демонстрация rename_analog_signals и rename_digital_signals ---


print(f"Каталог для этого шага: {os.path.abspath(base_source_cfg_dir)}")

# Должны быть заданы уже обработанные файлы csv!!
analog_csv_path = os.path.join(base_source_cfg_dir, "sorted_analog_signals_name.csv")
digital_csv_path = os.path.join(base_source_cfg_dir, "sorted_digital_signals_name.csv")


# Вызов rename_analog_signals
print("\nВызов rename_analog_signals...")
rename_analog_signals(source_dir=base_source_cfg_dir, csv_dir=analog_csv_path)
print("Функция rename_analog_signals завершена.")

# Вызов rename_digital_signals
print("\nВызов rename_digital_signals...")
rename_digital_signals(source_dir=base_source_cfg_dir, csv_dir=digital_csv_path)
print("Функция rename_digital_signals завершена.")


## 6: Проверка и очистка русских букв в файлах конфигурации

После всех преобразований необходимо убедиться, что в файлах конфигурации нет русских букв, которые могут помешать обработке данных на разных платформах.

Эта функция:
- Сканирует все `.cfg` файлы в поиске кириллицы
- Опционально применяет пользовательские правила замены (приоритет важен!)
- Генерирует детальный отчёт о результатах

**Параметры `replacement_rules`:**
- Список кортежей `(старое, новое)` в порядке приоритета
- Порядок — это приоритет выполнения!
- Если `None` или пуст, то только поиск без изменений

In [ ]:
# Импортируем функцию для проверки русских букв
from osc_tools.data_management.renaming import find_and_fix_russian_chars_in_cfg

source_dir = "D:\DataSet\__Open EE osc Dataset v1.2 — копия"

source_cfg_dir_for_russian = os.path.join(project_root, "raw_data","002_run_processing_oscillograms_guide", "(копия)depersonalized_ALL_OSC_v2.5","")

print("Функция find_and_fix_russian_chars_in_cfg успешно импортирована.")

### Вариант 1: Только поиск русских букв (без замены)

In [ ]:
# ОПЦИЯ 1: Сканирование русских букв (только поиск, без замены)
print("=" * 80)
print("ОПЦИЯ 1: Сканирование русских букв (только поиск)")
print("=" * 80)

result_search = find_and_fix_russian_chars_in_cfg(
    source_dir=source_dir,
    output_report_path=os.path.join(source_dir, "russian_chars_fixed_report.txt")
)

print(f"\nРезультаты поиска:")
print(f"  - Всего .cfg файлов сканировано: {result_search['total_files_scanned']}")
print(f"  - Файлов с русскими буквами: {len(result_search['files_with_russian'])}")
print(f"  - Ошибок: {len(result_search['errors'])}")

if result_search['files_with_russian']:
    print(f"\nФайлы с русскими буквами:")
    for fpath in result_search['files_with_russian'][:5]:  # Покажем первые 5
        print(f"  - {os.path.basename(fpath)}")
    if len(result_search['files_with_russian']) > 5:
        print(f"  ... и ещё {len(result_search['files_with_russian']) - 5} файлов")

ОПЦИЯ 1: Сканирование русских букв (только поиск)
Подсчитываем общее количество файлов в исходном каталоге...
Всего файлов: 102362, начинаем сканирование...


Поиск русских букв в .cfg: 100%|██████████| 102362/102362 [00:04<00:00, 22234.88it/s]


✓ Отчёт сохранён: D:\DataSet\__Open EE osc Dataset v1.2 — копия\russian_chars_fixed_report.txt

Сканирование завершено!
Всего .cfg файлов: 51175
Файлов с русскими буквами: 0
Ошибок: 0


Результаты поиска:
  - Всего .cfg файлов сканировано: 51175
  - Файлов с русскими буквами: 0
  - Ошибок: 0


### Вариант 2: Поиск и замена с приоритетом

Этот вариант применяет правила замены в заданном порядке приоритета. Очень важно задавать правила от более специфичных к более общим!

In [ ]:
# ОПЦИЯ 2: Сканирование и замена русских букв с приоритетом
print("\n" + "=" * 80)
print("ОПЦИЯ 2: Сканирование и замена русских букв (с приоритетом)")
print("=" * 80)

# Определяем правила замены (порядок = приоритет)
# Специфичные правила идут первыми, более общие — в конце
replacement_rules = [
    ('кA', 'kA'),      # Приоритет 1: замена ...
    ('кB', 'kV'),      # Приоритет 2: замена ...
    ('н-', 'N'),
    ('Nк', 'N'),       # ошибка от предыдущего шага
    ('н - к', 'N'),
    ('N (ф)', 'N'),
    ('ттнп', 'N'),
    ('Рђ', ' '),       # Какой-то шум
    ('лалј', 'C')      # Описка форматов
]

source_dir_1 = os.path.join(source_dir, "delete","")
source_dir_1 = source_dir

print("\nПрименяемые правила замены (по приоритету):")
for i, (old, new) in enumerate(replacement_rules, 1):
    print(f"  {i}. '{old}' → '{new}'")

result_fix = find_and_fix_russian_chars_in_cfg(
    source_dir=source_dir_1,
    output_report_path=os.path.join(source_dir_1, "russian_chars_fixed_report.txt"),
    replacement_rules=replacement_rules
)

print(f"\nРезультаты обработки:")
print(f"  - Всего .cfg файлов сканировано: {result_fix['total_files_scanned']}")
print(f"  - Файлов с русскими буквами найдено: {len(result_fix['files_with_russian'])}")
print(f"  - Файлов успешно обновлено: {len(result_fix['replacements_made'])}")
print(f"  - Ошибок: {len(result_fix['errors'])}")

if result_fix['replacements_made']:
    print(f"\nОбновленные файлы:")
    for fpath, count in list(result_fix['replacements_made'].items())[:5]:
        print(f"  - {os.path.basename(fpath)}: {count} замен")
    if len(result_fix['replacements_made']) > 5:
        print(f"  ... и ещё {len(result_fix['replacements_made']) - 5} файлов")

if result_fix['errors']:
    print(f"\nОшибки при обработке:")
    for err in result_fix['errors'][:3]:
        print(f"  - {err}")
    if len(result_fix['errors']) > 3:
        print(f"  ... и ещё {len(result_fix['errors']) - 3} ошибок")


ОПЦИЯ 2: Сканирование и замена русских букв (с приоритетом)

Применяемые правила замены (по приоритету):
  1. 'кA' → 'kA'
  2. 'кB' → 'kV'
  3. 'н-' → 'N'
  4. 'Nк' → 'N'
  5. 'н - к' → 'N'
  6. 'N (ф)' → 'N'
  7. 'ттнп' → 'N'
  8. 'Рђ' → ' '
  9. 'лалј' → 'C'
Подсчитываем общее количество файлов в исходном каталоге...
Всего файлов: 102362, начинаем сканирование...


Поиск русских букв в .cfg: 100%|██████████| 102362/102362 [00:04<00:00, 21985.54it/s]


✓ Отчёт сохранён: D:\DataSet\__Open EE osc Dataset v1.2 — копия\russian_chars_fixed_report.txt

Сканирование завершено!
Всего .cfg файлов: 51175
Файлов с русскими буквами: 45
Успешно обновлено файлов: 0
Ошибок: 0


Результаты обработки:
  - Всего .cfg файлов сканировано: 51175
  - Файлов с русскими буквами найдено: 45
  - Файлов успешно обновлено: 0
  - Ошибок: 0


# Внимание: удаление не универсальных имён

Лишние строки пока что удаляются через отдельную специализированную программу, не входящую в данный пакет, на основе ранее сформированных универсальных имён. После выполнения шага удаления неуниверсальных имён рекомендуется запустить проверку кириллицы в файлах `.cfg` (нижеследующий шаг), чтобы убедиться в отсутствии русских символов, мешающих дальнейшей обработке.

Планируется в будущем интегрировать логику выявления и удаления неуниверсальных имён в этот пакет.


### Итоговая проверка

Все файлы обработаны! Теперь данные готовы к использованию:

- ✓ Конфиденциальная информация удалена
- ✓ Даты изменения нормализованы
- ✓ Файлы сгруппированы по техническим параметрам
- ✓ Имена сигналов стандартизированы
- ✓ Русские буквы заменены на латиницу
- ✓ Comtrade файлы пересохранены с удалением не универсальных имён (пока что через стороннюю программу APScilloscope)

**Отчёты сохранены в:**
- `russian_chars_search_report.txt` — результаты поиска
- `russian_chars_fixed_report.txt` — результаты замены (если применялись правила)

Данные готовы для дальнейшей обработки и анализа!

## Дополнительные функции: инструкции и примеры использования

Ниже приведены краткие описания и примеры вызова функций, которые используются в процессе подготовки осциллограмм, но отсутствовали в основном примере. Перед запуском убедитесь, что вы задали `base_source_cfg_dir` и необходимые CSV/JSON-файлы.

1) `extract_frequencies(file_path, threshold=0.1, isPrintMessege=False)` — извлекает `f_network` и `f_rate`.

2) `rename_one_signals(source_dir, old_name, new_name)` — быстрый фикс одного имени сигнала.

3) `combining_databases_of_unique_codes(old_csv, new_csv, merged_csv, ...)` — объединение CSV со статистикой имён.

4) `combining_json_hash_table(source_dir, encoding='utf-8')` — объединение JSON-хеш-таблиц в один файл.

Примечания и рекомендации:
- Всегда сначала сделайте резервную копию исходных `.cfg`/`.dat` файлов перед массовыми операциями.
- Если вы не уверены в массивах замен, сначала запускайте функции в тестовом каталоге или используйте `isPrintMessege=True` для отладки.
- После объединения CSV/JSON рекомендуется визуально проверить результат (10–20 строк).

In [ ]:
# Пример 1: extract_frequencies
cfg_example = os.path.join(base_source_cfg_dir, 'example')
print('Путь к примеру cfg:', cfg_example)
if not os.path.exists(cfg_example):
    print("Примерный файл cfg не найден. Создайте или укажите свой.")
else:
    try:
        f_network, f_rate = extract_frequencies(file_path=cfg_example, threshold=0.1, isPrintMessege=True)
        print(f'network: {f_network}, rate: {f_rate}')
    except Exception as e:
        print('Ошибка при извлечении частот:', e)

In [ ]:
# Пример 2: rename_one_signals
old = 'Voltage_X1'
new = 'U | SubstationX-Line1 | phase:A'
print(f"Попытка переименовать '{old}' → '{new}' в каталоге {base_source_cfg_dir}")
try:
    rename_one_signals(source_dir=base_source_cfg_dir, old_name=old, new_name=new)
    print('Переименование выполнено (если были вхождения).')
except Exception as e:
    print('Ошибка при rename_one_signals:', e)

In [ ]:
# Пример 3: combining_databases_of_unique_codes
old_csv = os.path.join(base_source_cfg_dir, 'sorted_analog_signals_name.csv')
new_csv = os.path.join(project_root, 'other_changes', 'sorted_analog_signals_name_new.csv')
merged_csv = os.path.join(base_source_cfg_dir, 'merged_sorted_analog_signals.csv')

# Создадим небольшой демонстрационный new_csv, если его нет (необязательно в рабочем режиме)
if not os.path.exists(new_csv):
    os.makedirs(os.path.dirname(new_csv), exist_ok=True)
    with open(new_csv, 'w', encoding='utf-8') as f:
        f.write('Key,universal_code,Value\nSampleSignal,SampleSignalStd,1\n')
    print('Создан демонстрационный new_csv:', new_csv)

try:
    combining_databases_of_unique_codes(old_csv_file_path=old_csv, new_csv_file_path=new_csv, merged_csv_file_path=merged_csv)
    print('Объединение CSV выполнено. Результат в:', merged_csv)
except Exception as e:
    print('Ошибка при объединении CSV:', e)

In [ ]:
# Пример 4: combining_json_hash_table
json_src = os.path.join(project_root, 'raw_data', '001_run_search_oscillograms_guide', 'Output')
print('Папка для объединения json-хешей:', json_src)
if os.path.exists(json_src):
    try:
        combining_json_hash_table(source_dir=json_src)
        print('combine_hash_table.json создан в', json_src)
    except Exception as e:
        print('Ошибка при объединении json-хешей:', e)
else:
    print('Папка с json-хешами не найдена, пропускаем пример:', json_src)